In [ ]:
%matplotlib inline
import pandas as pd
from pandas import Series
import numpy as np
from functools import reduce
from sklearn.preprocessing import LabelEncoder
from keras.layers import Dense, Lambda
from keras.initializers import glorot_normal
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from numpy import array
from keras.utils import to_categorical
import tensorflow as tf
from sklearn.metrics import recall_score, precision_score, precision_recall_fscore_support
import sqlite3
import keras
from scipy.special import softmax
from sklearn.metrics import accuracy_score

In [ ]:
data = pd.read_csv("data/orig/cleaned_data_notEncoded.csv")

In [ ]:
data['label_away'] = data['label_home'].apply(lambda x: -1 * x)

data['home_won'] = data['label_home'].apply(lambda x: 1 if x == 1 else 0)
data['home_loss'] = data['label_home'].apply(lambda x: 1 if x == -1 else 0)
data['home_draw'] = data['label_home'].apply(lambda x: 1 if x == 0 else 0)

data['away_won'] = data['label_away'].apply(lambda x: 1 if x == 1 else 0)
data['away_loss'] = data['label_away'].apply(lambda x: 1 if x == -1 else 0)
data['away_draw'] = data['label_away'].apply(lambda x: 1 if x == 0 else 0)

In [ ]:
#match_data = pd.read_csv("data/orig/match_data.csv")
conn = sqlite3.connect('database.sqlite')
res = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
match_data = pd.read_sql_query("SELECT * FROM Match", conn)

In [ ]:
relevant_match_data = match_data[['season', 'match_api_id', 'date', 'home_team_api_id', 'away_team_api_id']]

In [ ]:
data_with_season_date = pd.merge(data, relevant_match_data, on='match_api_id')

In [ ]:
grouped_home = data_with_season_date.groupby(['season', 'home_team_api_id'])

In [ ]:
names = []
groups = []
for name, group in grouped_home:
    names += [name]
    groups += [group]
#     print(name)
#     print(group.shape)

In [ ]:
new_groups = []
for i in range(len(groups)):
    df = groups[i]
    name = names[i]
    df['home_win_record'] = df['home_won'].cumsum() - (df['home_won'].cumsum() > 0)
    df['home_loss_record'] = df['home_loss'].cumsum() - (df['home_loss'].cumsum() > 0)
    df['home_draw_record'] = df['home_draw'].cumsum() - (df['home_draw'].cumsum() > 0)
    df['season'] = name[0]
    df['home_team_api_id'] = name[1]
    new_groups += [df]

In [ ]:
df_home_stats = pd.concat(new_groups)

In [ ]:
df_home_stats[['label_home', 'home_win_record']].head()

In [ ]:
df_home_stats.columns.values

In [ ]:
grouped_away = df_home_stats.groupby(['season', 'away_team_api_id'])

In [ ]:
names = []
groups = []
for name, group in grouped_away:
    names += [name]
    groups += [group]
#     print(name)
#     print(group.shape)
# print(names[4])
# groups[1]

In [ ]:
new_groups = []
for i in range(len(groups)):
    df = groups[i]
    name = names[i]
    df['away_win_record'] = df['away_won'].cumsum() - (df['away_won'].cumsum() > 0)
    df['away_loss_record'] = df['away_loss'].cumsum() - (df['away_loss'].cumsum() > 0)
    df['away_draw_record'] = df['away_draw'].cumsum() - (df['away_draw'].cumsum() > 0)
    df['season'] = name[0]
    df['away_team_api_id'] = name[1]
    new_groups += [df]

In [ ]:
df_final = pd.concat(new_groups)

In [ ]:
df_final.shape

In [ ]:
df_final.columns.values

In [ ]:
df_final['season'].values

In [ ]:
df_final['season'] = pd.factorize(df_final['season'])[0]

In [ ]:
df_final = df_final.drop(columns=['label_away', 'home_won', 'home_loss', 'home_draw',
                                  'away_won', 'away_loss', 'away_draw', 'date'])

In [ ]:
desired_columns = ['season', 'home_win_record', 'home_loss_record', 'home_draw_record',
                  'away_win_record', 'away_loss_record', 'away_draw_record']

for i in range(1, 12):
    for status in ['home_player', 'away_player']:
        posX_name = status + '_X' + str(i)
        posY_name = status + '_Y' + str(i)
        
        rating_name = status + '_' + str(i) + '_' + 'overall_rating'
        desired_columns.extend([posX_name, posY_name, rating_name])

df_rel = df_final[desired_columns]
df_rel.columns.values

In [ ]:
df_rel.shape

In [ ]:
df_label = df_final['label_home']
df_label.shape

In [ ]:
X = df_rel.values
y = df_label.values

In [ ]:
y = [i + 1 for i in y]
y = array(y)
y = to_categorical(y)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1) 
y.shape

In [ ]:
def recall(y_true, y_pred):
    print(type(y_true))
    print(tf.keras.backend.get_value(y_true))
    recall_vals = recall_score(y_true, y_pred, average=None)
    recall_vals = tf.convert_to_tensor(recall_vals, dtype=tf.float32)
    return recall_vals



In [ ]:
def one_hot(a, num_classes):
    return np.squeeze(np.eye(num_classes)[a.reshape(-1)])

def createToyModel(input_dim):
    
    xavier_init = glorot_normal(seed=54)
    model = Sequential()
    # model.add(Dense(units=54, activation='elu', kernel_initializer=xavier_init))
    model.add(Dense(units=27, activation='elu', kernel_initializer=xavier_init))
    model.add(Dense(units=18, activation='elu', kernel_initializer=xavier_init))
    model.add(Dense(units=9, activation='elu', kernel_initializer=xavier_init))
    model.add(Dense(units=3, activation='softmax', kernel_initializer=xavier_init))
    decay_rate = 3e-4 / 30
    opt = Adam(lr=3e-4, decay=decay_rate)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    return model


# Be careful about overwriting weights in the cell below

In [ ]:
from sklearn.model_selection import StratifiedKFold
from keras.callbacks import ModelCheckpoint
from numpy.random import seed
from tensorflow import set_random_seed
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1).split(X_train, np.argmax(y_train, axis=1))

seed(5)
set_random_seed(10)
val_loss = []
val_acc = []
for i,f in enumerate(folds):
    fpath = 'seung_best'+str(i)+'.h5' # change me
    checkpoint = ModelCheckpoint(fpath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    train = f[0]
    val = f[1]

    X_fold_train, y_fold_train = X_train[train], y_train[train]
    X_fold_val, y_fold_val = X_train[val], y_train[val]
    
    model = createToyModel(X_train.shape[1])
    model.fit(x=X_fold_train, y=y_fold_train, batch_size=8, shuffle=True, epochs=30, 
    validation_data=(X_fold_val, y_fold_val), callbacks=callbacks_list)
    
    model.load_weights(fpath)
    loss, acc = model.evaluate(X_fold_val, y_fold_val)
    val_loss.append(loss)
    val_acc.append(acc)

In [ ]:
print('Val losses:', val_loss)
print('Val accs:', val_acc)
print('Avg. val acc', np.mean(val_acc))

# Load and evaluate

In [ ]:
train_loss = []
train_acc = []
predicted = []
model = createToyModel(X_train.shape[1])
model.fit(x=X_fold_train, y=y_fold_train, batch_size=8, shuffle=True, epochs=1, 
    validation_data=(X_fold_val, y_fold_val), callbacks=callbacks_list)

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1).split(X_train, np.argmax(y_train, axis=1))
for i,f in enumerate(folds):
    fpath = 'fixed_best'+str(i)+'.h5' # CHANGE ME
    train = f[0]
    val = f[1]

    X_fold_train, y_fold_train = X_train[train], y_train[train]
    X_fold_val, y_fold_val = X_train[val], y_train[val]

    
    
    model.load_weights(fpath)
    loss, acc = model.evaluate(X_fold_train, y_fold_train)
    y_pred = model.predict(x=X_test)
    predicted.append(y_pred)
    train_loss.append(loss)
    train_acc.append(acc)

In [ ]:
print(np.mean(np.array(val_acc) - np.array(train_acc)))

In [ ]:
df = pd.DataFrame(index=pd.Index(['Model ' + str(i) for i in range(5)] + ['Ensemble averaging']), columns=['precision', 'recall', 'f1', 'accuracy']).fillna(0)
for i, p in enumerate(predicted):
    p_score, r_score, f1_score, _ = precision_recall_fscore_support(y_test, one_hot(np.argmax(p, axis=1).astype(int), 3), average='weighted')
    acc = accuracy_score(np.argmax(y_test, axis=1), np.argmax(p, axis=1))
    df.loc['Model ' +str(i), 'precision'] = p_score
    df.loc['Model ' +str(i), 'recall'] = r_score
    df.loc['Model ' +str(i), 'f1'] = f1_score
    df.loc['Model ' +str(i), 'accuracy'] = acc
df

In [ ]:
# normalize predictions for all models
predicted_arr = np.array(predicted)
# for i in range(5):
#     for j in range(predicted_arr.shape[1]):
#         predicted_arr[i, j, :] /= np.linalg.norm(predicted_arr[i, j, :] )

In [ ]:
predicted_mean = np.mean(predicted_arr, axis=0)

In [ ]:
# plt.figure()
# plt.hist(np.argmax(predicted_mean, axis=1), color='r', alpha=0.5, label='pred')
# plt.hist(np.argmax(y_test, axis=1), color='yellow', alpha=0.3, label='true')
p_score, r_score, f1_score, _ = precision_recall_fscore_support(y_test, one_hot(np.argmax(predicted_mean, axis=1).astype(int), 3), average='weighted')
acc = accuracy_score(np.argmax(y_test, axis=1), np.argmax(predicted_mean, axis=1))
df.loc['Ensemble averaging', 'precision'] = p_score
df.loc['Ensemble averaging', 'recall'] = r_score
df.loc['Ensemble averaging', 'f1'] = f1_score
df.loc['Ensemble averaging', 'accuracy'] = acc

# df.to_csv('deeper_prediction_scores.csv') # BE CAREFUL ABOUT OVERWRITING
df

In [ ]:
# odds = pd.read_csv('odds.csv')